Risk workflow for snow and blizzard hazards 
 Blizzard  
 A blizzard is a severe storm condition defined by low temperature, sustained wind or frequent wind gust and considerable precipitating or blowing snow. For blizzard conditions we propose the use of following impact indicator: Tmean  ≤ 0 oC, Rs (snow amount) ≥ 10 cm and Wg (wind gust) ≥ 17 m/s ( Vajda et al., 2014). This impact indicator was defined taking into account the exposure of critical infrastructure, i.e., roads, rails, power lines, telecommunication to the hazard and is based on an extensive literature review, media reports, surveys conducted with European CI operators and case studies. 
 Heavy snowfall may cause many disruptions and impacts in various sectors; however, the impacts and consequences of this hazard depend on the affected sector, infrastructure and also preparedness of society that varies over Europe.  For example, already a few centimeters of snow can disrupt road traffic, but doesn’t normally cause any harm to energy infrastructure. Many weather services have three warning levels based on the severity of expected impacts, which are typically different for different sectors of infrastructure. There is a large variation in the national warning criteria or thresholds.
 Similarly to blizzard, the impact indicators for heavy snowfall were defined taking into account the exposure of critical infrastructure, i.e., roads, rails, power lines, telecommunication to the hazard and is based on an extensive literature review, media reports, surveys conducted with European CI operators and case studies. The qualitative description of the two-level thresholds are:
 1st threshold: Some adverse impacts are expected, their severity depends on the resilience of the system, transportation is mainly affected.
 2nd threshold: The weather phenomena are so severe that is likely that adverse impact will occur, CI system is seriously impacted.

In [ ]:
#| Load libraries
   
    #| In this notebook we will use the following Python libraries:\n",
    #| [os](https://docs.python.org/3/library/os.html) - To create directories and work with files\n",
    #|[pooch](https://www.fatiando.org/pooch/latest/index.html) - To download and unzip the data\n",
    #| [rasterio](https://rasterio.readthedocs.io/en/stable/) - To access and explore geospatial raster data in GeoTIFF format\n",
    #| [xarray](https://docs.xarray.dev/en/stable/) - To process the data and prepare it for damage calculation\n",
    #| [rioxarray](https://corteva.github.io/rioxarray/stable/) - Rasterio xarray extension - to make it easier to use GeoTIFF data with xarray\n",
    #| [damagescanner](https://damagescanner.readthedocs.io/en/latest/#) - To calculate the damage maps\n",
    #| [cartopy](https://scitools.org.uk/cartopy/docs/latest/) - To plot the maps\n",
    #| [matplotlib](https://matplotlib.org/) - For plotting as well"

import os
from glob import glob
import numpy as np
import rasterio

import xarray as xr
import rioxarray as rxr

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import xclim as xc



# Define thresholds 
lim_tas=0.
lim_pr=10.
lim_gust=17.
lim_snow = 10.
lim_snow6 = 6.
lim_snow25 = 25.
#lim_gust=7.


# Read a Data
# Unit U10 (m s-1)

# Read multiple NetCDF file once

u10 = xr.open_mfdataset('/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/10m_u_component_of_wind_DMIN_era5Land_202*.nc').u10


#u10 = xr.open_dataset("/Users/poladesu/D_Drive/Prog_FMI/dmin_CLIMAAX/10m_u_component_of_wind_DMIN_era5Land_202112.nc").u10

# Unit V10 (m s-1)
v10 = xr.open_mfdataset('/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/10m_v_component_of_wind_DMIN_era5Land_202*.nc').v10

# Calculate wind speed from U and V component (m/s)
wspd = np.sqrt((u10 * u10) + (v10 * v10))
wspd = wspd.assign_attrs(units="m/s", description="Wind Speed")

wspd1 = wspd.values[0:31, 200, 1500]

del u10, v10


# Read Temperature: Unit (K)
tas = xr.open_mfdataset('/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/2m_temperature_DMIN_era5Land_202*.nc').t2m

tas = tas.assign_attrs(units="K", description="Temperature data")

tas = xc.units.convert_units_to(tas, "degC")

tas1 = tas.values[0:31, 200, 1500]

# Read snow data 
snow = xr.open_mfdataset('/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/snow_depth_DMIN_era5Land_202*.nc').sde
# snow unit M to CM
snow = snow * 100
snow1 = snow.values[0:31, 200, 1500]

# Calculate the blizzard days 

BdayCount = (tas < lim_tas) * (snow > lim_snow) * (wspd > lim_gust) * 1

BdayCount_ana = BdayCount.resample(time="YS").sum(dim="time") # where M is for months

BdayCount_anaProb = (tas < lim_tas) * (snow > lim_snow) * (wspd > lim_gust).groupby('time.year').mean('time')

BdayCount_ana.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/BdayCount_ana.nc")
BdayCount_anaProb.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/BdayCount_anaProb.nc")




BdayCount_masked = BdayCount.where(BdayCount != 0.)

BdayCount_masked = BdayCount_masked.assign_attrs(units="number", long_name="blizzard days count")


BdayCount_masked = BdayCount_masked.to_dataset(name='count')


BdayCount_masked.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test5.nc")





snow6Count = (snow > lim_snow6) * 1

snow6Count_masked = snow6Count.where(snow6Count != 0.)

snow6Count_masked = snow6Count_masked.assign_attrs(units="number", long_name="snow days")


snow6Count_masked = snow6Count_masked.to_dataset(name='snow_days')


snow6Count_masked.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test15.nc")



snow25Count = (snow > lim_snow25) * 1

snow25Count_masked = snow25Count.where(snow25Count != 0.)

snow25Count_masked = snow25Count_masked.assign_attrs(units="number", long_name="snow days")


snow25Count_masked = snow25Count_masked.to_dataset(name='snow_days')


snow6Count_masked.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test15.nc")



snow25Count = (snow > lim_snow25) * 1

snow25Count_masked = snow25Count.where(snow25Count != 0.)

snow25Count_masked = snow25Count_masked.assign_attrs(units="number", long_name="snow days")


snow25Count_masked = snow25Count_masked.to_dataset(name='snow_days')


snow25Count_masked.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test25.nc")




monthly_sum = snow25Count_masked.resample(time="year").sum(dim="time") # where M is for months

monthly_sum.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test35.nc")

# Probability of occurence in %

snowProb = (snow > 25.).groupby('time.year').mean('time')

snowProb1 = snowProb.values[0, 200, 200]

snowProb.to_netcdf(path="/fmi/scratch/project_2004544/dmin_DtEarth/ERA5_Land_all/test45.nc")